In [1]:
#%load_ext lab_black

In [2]:
import numpy as np
import pandas as pd
import itertools as it
import ipyparallel as ipp

from random import normalvariate

import gc
import timeit
import sys
import os

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [3]:
# Revoking stored reported data
# https://ipython.org/ipython-doc/rel-0.12/config/extensions/storemagic.html

%store -r data_Averoy
%store -r data_Tingvoll
%store -r data_Surnadal
%store -r data_Sunndal
%store -r data_Vestnes
%store -r data_Laerdal

# Constructing hypothetical (synthetic) populations

In [5]:
# Setting up the cluster
cluster = ipp.Cluster(n=52)
cluster.start_cluster_sync()
rc = cluster.connect_client_sync()
rc.wait_for_engines(52)
rc.ids
dv = rc[:]

Using existing profile dir: '/Users/stigomholt/.ipython/profile_default'
Starting 52 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


<Cluster(cluster_id='1739795138-zmnb', profile='default', controller=<running>, engine_sets=['1739795139'])>

  0%|          | 0/52 [00:00<?, ?engine/s]

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51]

In [6]:
with dv.sync_imports():
    from random import normalvariate
    import numpy as np

importing normalvariate from random on engine(s)
importing numpy on engine(s)


In [7]:
a = 10
b = 5
dv.push({"a": a, "b": b})

<AsyncResult(_push): pending>

In [8]:
# To ensure the variables are on engine 0
rc[0]["a"]

10

In [9]:
rc[50]["b"]

5

### Calculate the initial sizes of all six population categories

In [10]:
%%px
def calculate_initial_size_of_each_category(population_data):
    total_number_calves = population_data[0] * population_data[3]
    total_number_adults = population_data[0] * (1 - population_data[3])
    total_number_stags = total_number_adults / (population_data[1] + 1)
    total_number_hinds = total_number_adults - total_number_stags

    female_calves_survivors = total_number_calves / 2
    male_calves_survivors = total_number_calves / 2

    young_hinds_survivors = total_number_hinds * population_data[4]
    adult_hinds_survivors = total_number_hinds - young_hinds_survivors

    young_stags_survivors = total_number_stags * population_data[5]
    adult_stags_survivors = total_number_stags - young_stags_survivors
    return (
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
    )

In [11]:
%%px
def winter_mortality_all_categories(
    female_calves, young_hinds, adult_hinds, male_calves, young_stags, adult_stags
):
    # Assumed winter survival parameters for all six population categories
    w_survival = [
        normalvariate(0.8, 0.04),
        normalvariate(0.92, 0.03),
        normalvariate(0.93, 0.03),
        normalvariate(0.8, 0.04),
        normalvariate(0.87, 0.04),
        normalvariate(0.93, 0.03),
    ]
    if w_survival[0] >= 1.0:
        fraction_female_calves_surviving_winter = 1.0
    else:
        fraction_female_calves_surviving_winter = w_survival[0]

    if w_survival[1] >= 1.0:
        fraction_young_hinds_surviving_winter = 1.0
    else:
        fraction_young_hinds_surviving_winter = w_survival[1]

    if w_survival[2] >= 1.0:
        fraction_adult_hinds_surviving_winter = 1.0
    else:
        fraction_adult_hinds_surviving_winter = w_survival[2]

    if w_survival[3] >= 1.0:
        fraction_male_calves_surviving_winter = 1.0
    else:
        fraction_male_calves_surviving_winter = w_survival[3]

    if w_survival[4] >= 1.0:
        fraction_young_stags_surviving_winter = 1.0
    else:
        fraction_young_stags_surviving_winter = w_survival[4]

    if w_survival[5] >= 1.0:
        fraction_adult_stags_surviving_winter = 1.0
    else:
        fraction_adult_stags_surviving_winter = w_survival[5]

    female_calves_survivors = female_calves * fraction_female_calves_surviving_winter
    young_hinds_survivors = young_hinds * fraction_young_hinds_surviving_winter
    adult_hinds_survivors = adult_hinds * fraction_adult_hinds_surviving_winter

    male_calves_survivors = male_calves * fraction_male_calves_surviving_winter
    young_stags_survivors = young_stags * fraction_young_stags_surviving_winter
    adult_stags_survivors = adult_stags * fraction_adult_stags_surviving_winter

    return (
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
        fraction_female_calves_surviving_winter,
        fraction_young_hinds_surviving_winter,
        fraction_adult_hinds_surviving_winter,
        fraction_male_calves_surviving_winter,
        fraction_young_stags_surviving_winter,
        fraction_adult_stags_surviving_winter,
    )

### Spring update of population categories 

In [12]:
%%px
def spring_update_of_categories(
    female_calves_survivors,
    young_hinds_survivors,
    adult_hinds_survivors,
    male_calves_survivors,
    young_stags_survivors,
    adult_stags_survivors,
):
    # Surviving female calves become one year old young hinds
    young_hinds = female_calves_survivors

    # Surviving young hinds born almost two years ago become adult hinds
    adult_hinds = young_hinds_survivors + adult_hinds_survivors

    # Surviving male calves become one year old young stags
    young_stags = male_calves_survivors

    # Surviving young stags born almost two years ago become adult stags
    adult_stags = young_stags_survivors + adult_stags_survivors
    return young_hinds, adult_hinds, young_stags, adult_stags

### Calculate number of female and male calves born in early summer

In [13]:
%%px
def early_summer_breeding(young_hinds_survivors, adult_hinds_survivors):

    # Assumed fertility parameters for young and adult hinds
    calves_per_young_hind = normalvariate(0.4, 0.05)

    calves_per_adult_hind = normalvariate(0.975, 0.008)
    if calves_per_adult_hind >= 1.0:
        calves_per_adult_hind = 1.0

    female_calves_born = (
        young_hinds_survivors * calves_per_young_hind / 2
        + adult_hinds_survivors * calves_per_adult_hind / 2
    )
    male_calves_born = female_calves_born
    return (
        female_calves_born,
        male_calves_born,
        calves_per_young_hind,
        calves_per_adult_hind,
    )

### Calculate putative emigration of stags

In [14]:
%%px
def emigration_of_stags(stags_per_hind_threshold, adult_stags, adult_hinds):
    """
    Does not account for the hypothesis that migration is primarily sported by
    the two years old stags. The way we deal with the emigration issue is highly
    idealized as we say that when the number of adult stags per adult hind is
    larger than a certain number, then stags will emigrate until the ratio falls
    below this number. The underlying hypothesis is that the stags are
    capable of making an assessment of their potential breeding success based on
    perceptual data during the winter and the spring and act on this information
    by use of a built in algorithm.
    """

    if stags_per_hind_threshold == 0 or adult_hinds < 1 or adult_stags < 1:
        number_of_migrants = 0
    else:
        number_of_migrants = adult_stags - stags_per_hind_threshold * adult_hinds
        if number_of_migrants <= 0:
            number_of_migrants = 0

    if number_of_migrants > 0:
        migration_percentage = 100 * number_of_migrants / adult_stags
    else:
        migration_percentage = 0

    adult_stags = adult_stags - number_of_migrants

    return adult_stags, number_of_migrants, migration_percentage

### Calculate number of individuals in each category surviving the hunting season

In [15]:
%%px
def transform_harvested_percentages_to_numbers(
    total_harvest,
    fraction_female_calves_harvested,
    fraction_male_calves_harvested,
    fraction_young_hinds_harvested,
    fraction_adult_hinds_harvested,
    fraction_young_stags_harvested,
    fraction_adult_stags_harvested,
):
    number_harvested_female_calves = np.round(
        np.multiply(np.array(fraction_female_calves_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_young_hinds = np.round(
        np.multiply(np.array(fraction_young_hinds_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_adult_hinds = np.round(
        np.multiply(np.array(fraction_adult_hinds_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_male_calves = np.round(
        np.multiply(np.array(fraction_male_calves_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_young_stags = np.round(
        np.multiply(np.array(fraction_young_stags_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_adult_stags = np.round(
        np.multiply(np.array(fraction_adult_stags_harvested), np.array(total_harvest))
        / 100
    )
    return (
        number_harvested_female_calves,
        number_harvested_young_hinds,
        number_harvested_adult_hinds,
        number_harvested_male_calves,
        number_harvested_young_stags,
        number_harvested_adult_stags,
    )

In [16]:
%%px
def survivors_after_hunting(
    number_harvested_female_calves,
    number_harvested_young_hinds,
    number_harvested_adult_hinds,
    number_harvested_male_calves,
    number_harvested_young_stags,
    number_harvested_adult_stags,
    female_calves_born,
    male_calves_born,
    young_hinds,
    adult_hinds,
    young_stags,
    adult_stags,
):
    female_calves_survivors = female_calves_born - number_harvested_female_calves
    young_hinds_survivors = young_hinds - number_harvested_young_hinds
    adult_hinds_survivors = adult_hinds - number_harvested_adult_hinds

    male_calves_survivors = male_calves_born - number_harvested_male_calves
    young_stags_survivors = young_stags - number_harvested_young_stags
    adult_stags_survivors = adult_stags - number_harvested_adult_stags

    if female_calves_survivors < 0:
        female_calves_survivors = 0
    if young_hinds_survivors < 0:
        young_hinds_survivors = 0
    if adult_hinds_survivors < 0:
        adult_hinds_survivors = 0

    if male_calves_survivors < 0:
        male_calves_survivors = 0
    if young_stags_survivors < 0:
        young_stags_survivors = 0
    if adult_stags_survivors < 0:
        adult_stags_survivors = 0
    return (
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
    )

### Project population development from one year to the next

In [17]:
%%px
def project_population_development(
    number_harvested_female_calves,
    number_harvested_young_hinds,
    number_harvested_adult_hinds,
    number_harvested_male_calves,
    number_harvested_young_stags,
    number_harvested_adult_stags,
    female_calves_survivors,
    young_hinds_survivors,
    adult_hinds_survivors,
    male_calves_survivors,
    young_stags_survivors,
    adult_stags_survivors,
    stags_per_hind_threshold,
    year,
):

    # Let Mother winter take her toll
    (
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
        fraction_female_calves_surviving_winter,
        fraction_young_hinds_surviving_winter,
        fraction_adult_hinds_surviving_winter,
        fraction_male_calves_surviving_winter,
        fraction_young_stags_surviving_winter,
        fraction_adult_stags_surviving_winter,
    ) = winter_mortality_all_categories(
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
    )

    # Let calves become young ones and young ones become adults
    young_hinds, adult_hinds, young_stags, adult_stags = spring_update_of_categories(
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
    )
    # Total population in the spring
    total_population_in_spring = young_hinds + adult_hinds + young_stags + adult_stags

    # Time for giving birth
    (
        female_calves_born,
        male_calves_born,
        calves_per_young_hind,
        calves_per_adult_hind,
    ) = early_summer_breeding(young_hinds_survivors, adult_hinds_survivors)

    # A tough decision made by stags
    adult_stags, number_of_migrants, migration_percentage = emigration_of_stags(
        stags_per_hind_threshold, adult_stags, adult_hinds
    )

    # Calculate ratio hinds per stag just before hunting season
    if (young_stags + adult_stags) > 1.0:
        hinds_per_stag_before_hunting = (young_hinds + adult_hinds) / (
            young_stags + adult_stags
        )
    else:
        hinds_per_stag_before_hunting = "NaN"

    # Total population just before hunting season starts
    total_population_before_hunting = (
        female_calves_born
        + young_hinds
        + adult_hinds
        + male_calves_born
        + young_stags
        + adult_stags
    )

    # Oh Deer, life is becoming dangerous
    (
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
    ) = survivors_after_hunting(
        number_harvested_female_calves,
        number_harvested_young_hinds,
        number_harvested_adult_hinds,
        number_harvested_male_calves,
        number_harvested_young_stags,
        number_harvested_adult_stags,
        female_calves_born,
        male_calves_born,
        young_hinds,
        adult_hinds,
        young_stags,
        adult_stags,
    )

    # Total population in December after end of hunting season
    total_population_after_hunting = (
        female_calves_survivors
        + young_hinds_survivors
        + adult_hinds_survivors
        + male_calves_survivors
        + young_stags_survivors
        + adult_stags_survivors
    )

    # Calculate ratio hinds per stag in December after end of hunting season
    if (young_stags_survivors + adult_stags_survivors) > 1:
        hinds_per_stag_after_hunting = (
            young_hinds_survivors + adult_hinds_survivors
        ) / (young_stags_survivors + adult_stags_survivors)
    else:
        hinds_per_stag_after_hunting = "NaN"

    return (
        total_population_in_spring,
        number_of_migrants,
        migration_percentage,
        hinds_per_stag_before_hunting,
        total_population_before_hunting,
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
        total_population_after_hunting,
        hinds_per_stag_after_hunting,
        fraction_female_calves_surviving_winter,
        fraction_young_hinds_surviving_winter,
        fraction_adult_hinds_surviving_winter,
        fraction_male_calves_surviving_winter,
        fraction_young_stags_surviving_winter,
        fraction_adult_stags_surviving_winter,
        calves_per_young_hind,
        calves_per_adult_hind,
    )

### Transform harvest percentages to numbers

In [18]:
def transform_harvested_percentages_to_numbers(
    total_harvest,
    fraction_female_calves_harvested,
    fraction_male_calves_harvested,
    fraction_young_hinds_harvested,
    fraction_adult_hinds_harvested,
    fraction_young_stags_harvested,
    fraction_adult_stags_harvested,
):
    number_harvested_female_calves = np.round(
        np.multiply(np.array(fraction_female_calves_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_young_hinds = np.round(
        np.multiply(np.array(fraction_young_hinds_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_adult_hinds = np.round(
        np.multiply(np.array(fraction_adult_hinds_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_male_calves = np.round(
        np.multiply(np.array(fraction_male_calves_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_young_stags = np.round(
        np.multiply(np.array(fraction_young_stags_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_adult_stags = np.round(
        np.multiply(np.array(fraction_adult_stags_harvested), np.array(total_harvest))
        / 100
    )
    return (
        number_harvested_female_calves,
        number_harvested_young_hinds,
        number_harvested_adult_hinds,
        number_harvested_male_calves,
        number_harvested_young_stags,
        number_harvested_adult_stags,
    )

### Scripts for L2_norm_seen_deer analysis

In [19]:
%%px
def L2_norm_seen_deer(
    population_log,
    empirical_observations,
    number_of_years,
):
    """
    Calculating the minimum L2 norm between synthetic data and
    reported data
    """
    import numpy as np
    # Establish the two time series to be compared
    seen_deer_e = np.array(empirical_observations)
    before_hunt_record = [x[18] for x in population_log]
    seen_deer_p = np.array(before_hunt_record)
    
    if seen_deer_p[-1] > 10:
        # Find the scaling factor that minimises RSS and do the scaling
        scaling_factor = np.sum(np.multiply(seen_deer_p, seen_deer_e)) / np.sum(
            np.multiply(seen_deer_p, seen_deer_p)
        )
    else:
        scaling_factor = 100.
    
    seen_deer_p_scaled = scaling_factor * seen_deer_p

    # Find minimum Residual Sum Square value
    L2_norm_min = (np.sum((seen_deer_p_scaled - seen_deer_e) ** 2))**(1/2)

    return scaling_factor, L2_norm_min

### Main ipy parallel script

In [20]:
@dv.remote(block=True)
def ipy_master():
    import numpy as np

    # Initial preparations
    array_top_hits = np.zeros((number_of_top_hits, number_of_years, 34))
    new_val = np.arange(10000, 10000 + number_of_top_hits, 1)
    scaling_factor = 500
    for i in range(array_top_hits.shape[0]):
        array_top_hits[i, -1, array_top_hits.shape[2] - 2] = scaling_factor
        array_top_hits[i, -1, array_top_hits.shape[2] - 1] = new_val[i]

    # Start going through states
    for state in init_chunk:
        # Run through for replicate number z
        for z in range(number_of_replicates):
            replicate_number = z

            population_log = []

            # Calculate size of each category in the initial population December 31 year -1
            (
                female_calves_survivors,
                young_hinds_survivors,
                adult_hinds_survivors,
                male_calves_survivors,
                young_stags_survivors,
                adult_stags_survivors,
            ) = calculate_initial_size_of_each_category(state)

            # Record initial sizes of the six categories
            female_calves_initially = female_calves_survivors
            young_hinds_initially = young_hinds_survivors
            adult_hinds_initially = adult_hinds_survivors
            male_calves_initially = male_calves_survivors
            young_stags_initially = young_stags_survivors
            adult_stags_initially = adult_stags_survivors

            # Run through all years in the time series for each initial state
            for j in range(number_of_years):
                (
                    total_population_in_spring,
                    number_of_migrants,
                    migration_percentage,
                    hinds_per_stag_before_hunting,
                    total_population_before_hunting,
                    female_calves_survivors,
                    young_hinds_survivors,
                    adult_hinds_survivors,
                    male_calves_survivors,
                    young_stags_survivors,
                    adult_stags_survivors,
                    total_population_after_hunting,
                    hinds_per_stag_after_hunting,
                    fraction_female_calves_surviving_winter,
                    fraction_young_hinds_surviving_winter,
                    fraction_adult_hinds_surviving_winter,
                    fraction_male_calves_surviving_winter,
                    fraction_young_stags_surviving_winter,
                    fraction_adult_stags_surviving_winter,
                    calves_per_young_hind,
                    calves_per_adult_hind,
                ) = project_population_development(
                    number_harvested_female_calves[j],
                    number_harvested_young_hinds[j],
                    number_harvested_adult_hinds[j],
                    number_harvested_male_calves[j],
                    number_harvested_young_stags[j],
                    number_harvested_adult_stags[j],
                    female_calves_survivors,
                    young_hinds_survivors,
                    adult_hinds_survivors,
                    male_calves_survivors,
                    young_stags_survivors,
                    adult_stags_survivors,
                    state[2],
                    j,
                )

                new_record = [
                    state[0],
                    state[1],
                    state[2],
                    female_calves_initially,
                    young_hinds_initially,
                    adult_hinds_initially,
                    male_calves_initially,
                    young_stags_initially,
                    adult_stags_initially,
                    replicate_number,
                    first_year + j,
                    female_calves_survivors,
                    young_hinds_survivors,
                    adult_hinds_survivors,
                    male_calves_survivors,
                    young_stags_survivors,
                    adult_stags_survivors,
                    total_population_in_spring,
                    total_population_before_hunting,
                    total_population_after_hunting,
                    hinds_per_stag_before_hunting,
                    hinds_per_stag_after_hunting,
                    migration_percentage,
                    number_of_migrants,
                    fraction_female_calves_surviving_winter,
                    fraction_young_hinds_surviving_winter,
                    fraction_adult_hinds_surviving_winter,
                    fraction_male_calves_surviving_winter,
                    fraction_young_stags_surviving_winter,
                    fraction_adult_stags_surviving_winter,
                    calves_per_young_hind,
                    calves_per_adult_hind,
                ]

                population_log.append(new_record)

            # Do L2_norm fitting of the number_of_years population trajectory
            if municipality == "Surnadal":
                scaling_factor, L2_value = L2_norm_seen_deer(
                    population_log,
                    seen_deer_obs_farmland,
                    number_of_years,
                )
            else:
                scaling_factor, L2_value = L2_norm_seen_deer(
                    population_log,
                    seen_deer_obs,
                    number_of_years,
                )

            # Find max L2 norm in array_top_hits and exchange if better fit
            if L2_value < array_top_hits[np.argmax(array_top_hits[:, -1, -1]), -1, -1]:
                # Add scaling_factor and L2 value at the end of each record in population_log
                # and exchange number_of_years records
                for k in range(len(population_log)):
                    population_log[k].extend([scaling_factor, L2_value])
                    array_top_hits[
                        np.argmax(array_top_hits[:, -1, -1]), k, :
                    ] = population_log[k]

    return array_top_hits

In [21]:
%%time
    
# Number of replicates of each initial condition 
number_of_replicates = 50 #1 
number_of_top_hits = 2000 # from each engine
dv.push(dict(number_of_replicates = number_of_replicates, number_of_top_hits = number_of_top_hits))

# To ensure the variables are on engines 
#print(rc[0]['number_of_replicates'],rc[50]['number_of_top_hits']) 

# Using all six municipalties
municipalities = ["Averoy", "Tingvoll", "Surnadal", "Sunndal", "Vestnes", "Laerdal"]
data_municipality = [
    data_Averoy,
    data_Tingvoll,
    data_Surnadal,
    data_Sunndal,
    data_Vestnes,
    data_Laerdal,
]

for q in range(len(municipalities)):
    [
        municipality,
        first_year,
        last_year,
        number_of_years,
        years,
        seen_deer_obs,
        seen_deer_obs_forest,
        seen_deer_obs_farmland,
        hinds_per_stag_obs,
        total_harvest,
        total_quota,
        fraction_female_calves_harvested,
        fraction_young_hinds_harvested,
        fraction_adult_hinds_harvested,
        fraction_male_calves_harvested,
        fraction_young_stags_harvested,
        fraction_adult_stags_harvested,
        spring_counts,
    ] = data_municipality[q]
    
    dv.push(dict(municipality = municipality ,
            first_year = first_year,
            last_year = last_year,
            number_of_years = number_of_years,
            years = years,
            seen_deer_obs = seen_deer_obs,
            seen_deer_obs_forest = seen_deer_obs_forest,
            seen_deer_obs_farmland = seen_deer_obs_farmland,
            hinds_per_stag_obs = hinds_per_stag_obs,
            total_harvest = total_harvest,
            fraction_female_calves_harvested = fraction_female_calves_harvested,
            fraction_young_hinds_harvested = fraction_young_hinds_harvested,
            fraction_adult_hinds_harvested = fraction_adult_hinds_harvested,
            fraction_male_calves_harvested = fraction_male_calves_harvested,
            fraction_young_stags_harvested = fraction_young_stags_harvested,
            fraction_adult_stags_harvested = fraction_adult_stags_harvested,
            spring_counts = spring_counts))

    # To ensure the variables are on engines 
    #print(rc[5]['years'],rc[45]['total_harvest']) 
    
    # Setting up all initial conditions to be run with calibrated init_pop
    # Make all combinations of the six initial conditions
    #initial_population = np.arange(init_pop_range[q][0],init_pop_range[q][1], 10)
    initial_population = np.arange(900, 4000, 10)
    initial_hinds_per_stag = np.array([1.7]) # uses the inverse of the estimated stags_per_hind threshold
    #initial_hinds_per_stag = np.arange(1.3, 2.2, 0.1)
    init_frac_calves = np.arange(0.2, 0.355, 0.01)
    init_frac_y_hinds = np.arange(0.09, 0.275, 0.01)  # of total number hinds
    init_frac_y_stags = np.arange(0.18, 0.545, 0.02)  # of total number stags
    stags_per_hind_threshold = np.array([0.58])  # not really initial condition, but initial premise
    #stags_per_hind_threshold = np.arange(0.5, 0.8, 0.05)

    all_initial_states = np.array(
        np.meshgrid(
            initial_population,
            initial_hinds_per_stag,
            stags_per_hind_threshold,
            init_frac_calves,
            init_frac_y_hinds,
            init_frac_y_stags,
        )
    ).T.reshape(-1, 6)

    dv.scatter("init_chunk", all_initial_states)
    # To ensure the variables are on engines 
    #check = rc[0]['init_chunk']
    #print("len(init_chunk):",len(check)) 
    
    # Transform reported hunting percentages for all years to numbers
    (
        number_harvested_female_calves,
        number_harvested_young_hinds,
        number_harvested_adult_hinds,
        number_harvested_male_calves,
        number_harvested_young_stags,
        number_harvested_adult_stags,
    ) = transform_harvested_percentages_to_numbers(
        total_harvest,
        fraction_female_calves_harvested,
        fraction_male_calves_harvested,
        fraction_young_hinds_harvested,
        fraction_adult_hinds_harvested,
        fraction_young_stags_harvested,
        fraction_adult_stags_harvested,
    )

    dv.push(dict(number_harvested_female_calves = number_harvested_female_calves,
        number_harvested_young_hinds = number_harvested_young_hinds,
        number_harvested_adult_hinds= number_harvested_adult_hinds,
        number_harvested_male_calves = number_harvested_male_calves,
        number_harvested_young_stags= number_harvested_young_stags,
        number_harvested_adult_stags = number_harvested_adult_stags))
    
    # To ensure the variables are on engines 
    #print(rc[10]['number_harvested_young_hinds'],rc[45]['number_harvested_adult_stags']) 
    
    # Getting array_top_hits from all engines in a 3d array that needs to be flattened
    array_top_hits = np.array(ipy_master())
    aggregated_arr = array_top_hits.reshape(-1, array_top_hits.shape[-1])
          

    # Make pandas dataframe for given municipality
    column_values = [
        "init_pop",
        "init_hps",
        "sh_threshold",
        "i_fcalves",
        "i_yhinds",
        "i_ahinds",
        "i_mcalves",
        "i_ystags",
        "i_astags",
        "rep_num",
        "obs_year",
        "f_calves",
        "y_hinds",
        "a_hinds",
        "m_calves",
        "y_stags",
        "a_stags",
        "spring_pop",
        "tot_pop_bh",
        "tot_pop_ah",
        "hps_bh",
        "hps_ah",
        "mig%",
        "num_migs",
        "ws_fc",
        "ws_yh",
        "ws_ah",
        "ws_mc",
        "ws_ys",
        "ws_as",
        "c_yh",
        "c_ah",
        "scaling",
        "L2_norm",
    ]
    pop_data = pd.DataFrame(data=aggregated_arr, columns=column_values)
    pop_data = pop_data.astype(
        {
            "init_pop": "int",
            "rep_num": "int",
            "obs_year": "int",
            "i_fcalves": "int",
            "i_yhinds": "int",
            "i_ahinds": "int",
            "i_mcalves": "int",
            "i_ystags": "int",
            "i_astags": "int",
            "f_calves": "int",
            "y_hinds": "int",
            "a_hinds": "int",
            "m_calves": "int",
            "y_stags": "int",
            "a_stags": "int",
            "spring_pop": "int",
            "tot_pop_bh": "int",
            "tot_pop_ah": "int",
            "num_migs": "int",
        }
    ).round(14)

    # Sort and trim the data frame
    #pop_data_sorted = pop_data.sort_values(by=["L2_norm"]).reset_index(drop=True)
    pop_data_sorted = pop_data.sort_values(['L2_norm','obs_year'],ascending=[True, True]).reset_index(drop=True)
        
    # Storing all top hits to allow variation in the number of top hits later on
    save_dir = "synthetic_data/"
    pop_data_sorted.to_pickle(
        save_dir + "top_hits_" + municipality + "_seen_deer_stoch_par_fixed_sph_" + str(number_of_replicates) + ".pkl"
    )

    print("Municipality = ", municipality)
    pd.set_option('display.max_columns', None)
    display(pop_data_sorted)
    print()
    print()

Municipality =  Averoy


,init_pop,init_hps,sh_threshold,i_fcalves,i_yhinds,i_ahinds,i_mcalves,i_ystags,i_astags,rep_num,obs_year,f_calves,y_hinds,a_hinds,m_calves,y_stags,a_stags,spring_pop,tot_pop_bh,tot_pop_ah,hps_bh,hps_ah,mig%,num_migs,ws_fc,ws_yh,ws_ah,ws_mc,ws_ys,ws_as,c_yh,c_ah,scaling,L2_norm
0,1410,1.7,0.58,218,104,508,218,72,288,44,2007,204,134,512,200,165,298,1238,1714,1516,1.402613,1.392546,2.200323,7,0.755961,0.928779,0.898157,0.875608,0.830531,0.930811,0.396855,0.975153,0.000734,0.797918
1,1410,1.7,0.58,218,104,508,218,72,288,44,2008,211,136,538,213,103,312,1303,1742,1514,1.574011,1.624630,14.885085,58,0.876110,0.914207,0.895591,0.727883,0.803212,0.881341,0.395016,0.980079,0.000734,0.797918
2,1410,1.7,0.58,218,104,508,218,72,288,44,2009,213,91,527,216,152,342,1311,1803,1544,1.489083,1.250081,5.516999,20,0.792808,0.930509,0.887926,0.787427,0.845625,0.910143,0.347638,0.977654,0.000734,0.797918
3,1410,1.7,0.58,218,104,508,218,72,288,44,2010,212,97,488,210,139,288,1368,1717,1438,1.444453,1.367425,31.222840,145,0.794088,0.936575,0.886672,0.828564,0.869943,0.974913,0.453221,0.973627,0.000734,0.797918
4,1410,1.7,0.58,218,104,508,218,72,288,44,2011,216,86,500,216,140,294,1282,1724,1454,1.411236,1.350489,15.998100,61,0.730342,0.946556,0.956108,0.863306,0.972826,0.866566,0.493876,0.979969,0.000734,0.797918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1767995,1270,1.7,0.58,209,133,401,209,170,144,26,2019,177,88,441,180,108,263,1168,1593,1260,1.510956,1.423399,8.968370,30,0.834326,0.942853,0.923433,0.775501,0.946720,0.885510,0.418842,0.965213,0.000800,0.973187
1767996,1270,1.7,0.58,209,133,401,209,170,144,26,2020,161,48,395,171,100,251,1154,1547,1129,1.445262,1.261142,16.379066,57,0.824659,0.901426,0.956878,0.872391,0.828896,0.979643,0.480609,0.973245,0.000800,0.973187
1767997,1270,1.7,0.58,209,133,401,209,170,144,26,2021,117,40,277,107,82,206,1000,1265,832,1.525511,1.100704,29.272560,96,0.878648,0.908754,0.909661,0.719842,0.913303,0.951617,0.337253,0.964309,0.000800,0.973187
1767998,1270,1.7,0.58,209,133,401,209,170,144,26,2022,82,30,196,81,58,133,739,889,581,1.508133,1.184479,38.952617,105,0.812616,0.922156,0.892881,0.808380,0.875585,0.962505,0.356785,0.978792,0.000800,0.973187




Municipality =  Tingvoll


,init_pop,init_hps,sh_threshold,i_fcalves,i_yhinds,i_ahinds,i_mcalves,i_ystags,i_astags,rep_num,obs_year,f_calves,y_hinds,a_hinds,m_calves,y_stags,a_stags,spring_pop,tot_pop_bh,tot_pop_ah,hps_bh,hps_ah,mig%,num_migs,ws_fc,ws_yh,ws_ah,ws_mc,ws_ys,ws_as,c_yh,c_ah,scaling,L2_norm
0,2710,1.7,0.58,379,196,1031,379,144,578,46,2007,456,226,1076,434,249,577,2443,3478,3021,1.544407,1.575118,0.000000,0,0.832101,0.932248,0.953625,0.821992,0.870438,0.903952,0.425621,0.972683,0.00016,0.192051
1,2710,1.7,0.58,379,196,1031,379,144,578,46,2008,453,236,1060,429,291,627,2630,3611,3097,1.456372,1.410618,8.211782,61,0.733850,0.943180,0.905106,0.820897,0.863193,0.926595,0.414854,0.979056,0.00016,0.192051
2,2710,1.7,0.58,379,196,1031,379,144,578,46,2009,402,261,993,398,288,582,2638,3481,2927,1.484146,1.440954,18.039807,145,0.777917,0.876713,0.877441,0.801900,0.851610,0.887002,0.382454,0.976899,0.00016,0.192051
3,2710,1.7,0.58,379,196,1031,379,144,578,46,2010,358,225,953,358,226,569,2470,3267,2691,1.500599,1.482472,16.327910,124,0.759522,0.879462,0.877809,0.749874,0.815550,0.907612,0.383658,0.955878,0.00016,0.192051
4,2710,1.7,0.58,379,196,1031,379,144,578,46,2011,374,234,896,364,245,518,2387,3152,2633,1.490707,1.481763,17.790730,129,0.887368,0.878036,0.875855,0.856758,0.862278,0.938045,0.341044,0.990572,0.00016,0.192051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1767995,2310,1.7,0.58,265,268,851,265,303,355,30,2019,322,173,837,314,204,476,2151,2967,2328,1.493493,1.485269,5.377076,32,0.838221,0.921874,0.924541,0.834660,0.920719,0.918091,0.399233,0.972360,0.00018,0.227622
1767996,2310,1.7,0.58,265,268,851,265,303,355,30,2020,327,135,779,300,154,458,2060,2841,2156,1.510031,1.494289,9.551961,57,0.800103,0.931141,0.937067,0.791057,0.884036,0.894734,0.449789,0.976101,0.00018,0.227622
1767997,2310,1.7,0.58,265,268,851,265,303,355,30,2021,294,146,747,281,154,436,1988,2756,2061,1.509235,1.512110,7.627185,43,0.820510,0.937899,0.992418,0.840182,0.873910,0.940299,0.485423,0.969292,0.00018,0.227622
1767998,2310,1.7,0.58,265,268,851,265,303,355,30,2022,251,114,672,268,161,397,1826,2456,1866,1.473863,1.408106,15.535080,87,0.733656,0.905898,0.919551,0.807194,0.889051,0.974904,0.412069,0.964651,0.00018,0.227622




Municipality =  Surnadal


...



Municipality =  Vestnes


,init_pop,init_hps,sh_threshold,i_fcalves,i_yhinds,i_ahinds,i_mcalves,i_ystags,i_astags,rep_num,obs_year,f_calves,y_hinds,a_hinds,m_calves,y_stags,a_stags,spring_pop,tot_pop_bh,tot_pop_ah,hps_bh,hps_ah,mig%,num_migs,ws_fc,ws_yh,ws_ah,ws_mc,ws_ys,ws_as,c_yh,c_ah,scaling,L2_norm
0,2220,1.7,0.58,366,252,683,366,99,451,29,2007,271,240,773,266,246,415,1975,2613,2214,1.431736,1.530903,9.558844,51,0.816177,0.943544,0.871214,0.839819,0.839437,1.000000,0.418981,0.988552,0.000248,0.358355
1,2220,1.7,0.58,366,252,683,366,99,451,29,2008,306,129,843,323,149,462,1951,2695,2214,1.491084,1.590205,8.178999,48,0.745783,0.920354,0.924434,0.828273,0.826644,0.932363,0.446194,0.970401,0.000248,0.358355
2,2220,1.7,0.58,366,252,683,366,99,451,29,2009,321,184,761,318,204,412,1982,2738,2202,1.458497,1.535434,8.453691,47,0.843002,0.916138,0.914352,0.838365,0.842995,0.946835,0.364567,0.985959,0.000248,0.358355
3,2220,1.7,0.58,366,252,683,366,99,451,29,2010,273,168,742,271,157,415,1904,2608,2028,1.513468,1.592488,8.803351,48,0.767075,0.898103,0.925849,0.732091,0.869450,0.913380,0.416408,0.969635,0.000248,0.358355
4,2220,1.7,0.58,366,252,683,366,99,451,29,2011,295,151,748,291,188,389,1817,2476,2065,1.431117,1.558580,9.102738,48,0.782641,0.888125,0.912582,0.913123,0.917703,0.924605,0.327093,0.970681,0.000248,0.358355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1767995,1920,1.7,0.58,230,202,716,230,248,291,40,2019,319,190,812,285,172,448,2106,2857,2229,1.500734,1.615356,9.723131,58,0.879204,0.989212,0.895239,0.806456,0.930331,0.958556,0.388505,0.971437,0.000225,0.415407
1767996,1920,1.7,0.58,230,202,716,230,248,291,40,2020,314,178,836,306,135,448,1998,2820,2220,1.589066,1.737618,0.000000,0,0.820675,0.948869,0.964477,0.820952,0.853458,0.870020,0.340327,0.970274,0.000225,0.415407
1767997,1920,1.7,0.58,230,202,716,230,248,291,40,2021,341,169,829,325,141,441,2001,2866,2248,1.602945,1.712967,0.000000,0,0.804658,0.921697,0.974569,0.770978,0.876400,0.922639,0.429366,0.974585,0.000225,0.415407
1767998,1920,1.7,0.58,230,202,716,230,248,291,40,2022,335,225,812,317,204,443,1985,2784,2338,1.485577,1.601077,0.381125,2,0.791408,0.905395,0.918612,0.820900,0.904147,0.917049,0.417859,0.968014,0.000225,0.415407




Municipality =  Laerdal


,init_pop,init_hps,sh_threshold,i_fcalves,i_yhinds,i_ahinds,i_mcalves,i_ystags,i_astags,rep_num,obs_year,f_calves,y_hinds,a_hinds,m_calves,y_stags,a_stags,spring_pop,tot_pop_bh,tot_pop_ah,hps_bh,hps_ah,mig%,num_migs,ws_fc,ws_yh,ws_ah,ws_mc,ws_ys,ws_as,c_yh,c_ah,scaling,L2_norm
0,1840,1.7,0.58,248,93,752,248,89,407,31,2007,299,167,669,294,148,370,1611,2290,1950,1.512574,1.612661,3.942158,18,0.806365,0.940072,0.891776,0.780886,0.873994,0.931110,0.408149,0.986131,0.000104,0.142273
1,1840,1.7,0.58,248,93,752,248,89,407,31,2008,286,208,672,293,203,379,1740,2347,2043,1.453551,1.512987,10.759455,53,0.795792,0.884915,0.914923,0.835387,0.929370,0.962032,0.384782,0.984946,0.000104,0.142273
2,1840,1.7,0.58,248,93,752,248,89,407,31,2009,265,198,664,267,178,386,1764,2299,1960,1.502523,1.527745,19.476843,106,0.838881,0.866385,0.854966,0.765516,0.868523,0.970138,0.396665,0.988776,0.000104,0.142273
3,1840,1.7,0.58,248,93,752,248,89,407,31,2010,264,146,669,255,154,376,1661,2251,1868,1.501798,1.537006,10.076217,49,0.748586,0.940963,0.876140,0.741839,0.859263,0.886647,0.408972,0.967623,0.000104,0.142273
4,1840,1.7,0.58,248,93,752,248,89,407,31,2011,278,159,673,267,155,369,1669,2259,1904,1.499242,1.587678,12.948625,65,0.786619,0.957560,0.915705,0.799350,0.893615,0.966633,0.421375,0.972661,0.000104,0.142273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1767995,1780,1.7,0.58,284,205,556,284,107,340,40,2019,271,157,703,260,146,430,1763,2441,1970,1.513015,1.493935,5.037941,25,0.809727,0.924641,0.938534,0.760699,0.847228,0.876330,0.425470,0.966273,0.000107,0.157880
1767996,1780,1.7,0.58,284,205,556,284,107,340,40,2020,270,153,687,257,149,410,1755,2375,1928,1.501257,1.503292,15.054252,81,0.778244,0.903614,0.925073,0.802627,0.881834,0.959184,0.472383,0.974409,0.000107,0.157880
1767997,1780,1.7,0.58,284,205,556,284,107,340,40,2021,263,130,677,234,114,407,1746,2359,1827,1.518334,1.546154,15.374425,83,0.783485,0.903942,0.947861,0.783016,0.884270,0.998894,0.441511,0.974872,0.000107,0.157880
1767998,1780,1.7,0.58,284,205,556,284,107,340,40,2022,264,131,678,253,105,381,1620,2236,1815,1.531619,1.661348,9.827823,47,0.774131,0.917358,0.927981,0.800750,0.861543,0.937999,0.413332,0.977202,0.000107,0.157880




CPU times: user 3min 5s, sys: 7min 50s, total: 10min 56s
Wall time: 1h 53min 58s


In [22]:
# stop cluster syncronously
cluster.stop_cluster_sync()

Stopping controller
Controller stopped: {'exit_code': 0, 'pid': 69183, 'identifier': 'ipcontroller-1739795138-zmnb-69179'}
Stopping engine(s): 1739795139
engine set stopped 1739795139: {'engines': {'0': {'exit_code': 0, 'pid': 69196, 'identifier': '0'}, '1': {'exit_code': 0, 'pid': 69202, 'identifier': '1'}, '2': {'exit_code': 0, 'pid': 69203, 'identifier': '2'}, '3': {'exit_code': 0, 'pid': 69205, 'identifier': '3'}, '4': {'exit_code': 0, 'pid': 69207, 'identifier': '4'}, '5': {'exit_code': 0, 'pid': 69210, 'identifier': '5'}, '6': {'exit_code': 0, 'pid': 69213, 'identifier': '6'}, '7': {'exit_code': 0, 'pid': 69217, 'identifier': '7'}, '8': {'exit_code': 0, 'pid': 69220, 'identifier': '8'}, '9': {'exit_code': 0, 'pid': 69224, 'identifier': '9'}, '10': {'exit_code': 0, 'pid': 69231, 'identifier': '10'}, '11': {'exit_code': 0, 'pid': 69237, 'identifier': '11'}, '12': {'exit_code': 0, 'pid': 69243, 'identifier': '12'}, '13': {'exit_code': 0, 'pid': 69249, 'identifier': '13'}, '14': {'ex

In [23]:
cluster._is_running()

False

# Testing the sanity of the results

In [24]:
# Revoking top hits frames
save_dir = "synthetic_data/"
# With option for emigration
top_hits_Averoy_seen_deer = pd.read_pickle(
    save_dir
    + "top_hits_Averoy_seen_deer_stoch_par_fixed_sph_"
    + str(number_of_replicates)
    + ".pkl"
)
top_hits_Tingvoll_seen_deer = pd.read_pickle(
    save_dir
    + "top_hits_Tingvoll_seen_deer_stoch_par_fixed_sph_"
    + str(number_of_replicates)
    + ".pkl"
)
top_hits_Surnadal_seen_deer = pd.read_pickle(
    save_dir
    + "top_hits_Surnadal_seen_deer_stoch_par_fixed_sph_"
    + str(number_of_replicates)
    + ".pkl"
)
top_hits_Sunndal_seen_deer = pd.read_pickle(
    save_dir
    + "top_hits_Sunndal_seen_deer_stoch_par_fixed_sph_"
    + str(number_of_replicates)
    + ".pkl"
)
top_hits_Vestnes_seen_deer = pd.read_pickle(
    save_dir
    + "top_hits_Vestnes_seen_deer_stoch_par_fixed_sph_"
    + str(number_of_replicates)
    + ".pkl"
)
top_hits_Laerdal_seen_deer = pd.read_pickle(
    save_dir
    + "top_hits_Laerdal_seen_deer_stoch_par_fixed_sph_"
    + str(number_of_replicates)
    + ".pkl"
)

top_hits_seen_deer = [
    top_hits_Averoy_seen_deer,
    top_hits_Tingvoll_seen_deer,
    top_hits_Surnadal_seen_deer,
    top_hits_Sunndal_seen_deer,
    top_hits_Vestnes_seen_deer,
    top_hits_Laerdal_seen_deer,
]

In [25]:
save_dir = "synthetic_data/"
municipalities = ["Averoy", "Tingvoll", "Surnadal", "Sunndal", "Vestnes", "Laerdal"]
for q in range(len(top_hits_seen_deer)):
    # Read through and test for consistent periodicity regarding years
    # which checks that there are no conflicts associated with the sorting of the data frames
    counter = 0
    for i in range(0, len(top_hits_seen_deer[q]), number_of_years):
        for j in range(number_of_years):
            if top_hits_seen_deer[q]["obs_year"][i + j] != (2007 + j):
                # print('i:', i, 'j:', j, 'obs_year value:', top_hits_seen_deer[q]['obs_year'][i+j])
                counter = counter + 1
    print("Number of troubles in " + municipalities[q] + ":", counter)

Number of troubles in Averoy: 0
Number of troubles in Tingvoll: 0
Number of troubles in Surnadal: 0
Number of troubles in Sunndal: 0
Number of troubles in Vestnes: 0
Number of troubles in Laerdal: 0
